여기서 코드 돌아가는 것 확인하고 해당 파일 삭제 후 Final 제출 예정

In [ ]:
from nutri_ratio import format_recommendations
import pickle
from PIL import Image
import gradio as gr
from io import BytesIO
import pandas as pd
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import matplotlib.pyplot as plt

# Azure Custom Vision 설정
prediction_endpoint = "https://5a050project11-prediction.cognitiveservices.azure.com"
prediction_key = "2FKFWm82wp3KqlB4R3rCrhihILJyeJt42yU4QFE1Qm7Jz6eC3lRIJQQJ99AJACL93NaXJ3w3AAAIACOGsPzG"
project_id = "48948b9f-9c52-46ea-a6ae-43e46dcee2b3"
model_name = "FoodSyncFinal"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

# CSS 스타일링 추가
custom_css = """
    @import url('https://fonts.googleapis.com/css2?family=Gmarket+Sans:wght@400;700&display=swap');
    * { font-family: 'Gmarket Sans', sans-serif; }
    #title {
        text-align: center;
        color: #4CAF50;
        font-size: 2em;
        font-weight: bold;
        margin-bottom: 20px;
        margin-top: 15%;  /* 위쪽 여백을 추가하여 중앙으로 내림 */
    }
    #start-button {
        background-color: #4CAF50;
        color: white;
        font-size: 1.5em;
        font-weight: bold;
        border: none;
        cursor: pointer;
        padding: 10px 20px;
        margin-top: 10px;  /* 버튼과 제목 사이의 간격 조정 */
    }
    .centered-content {
        display: flex;
        flex-direction: column;
        align-items: center;
        justify-content: center;
        height: 100vh;  /* 화면의 전체 높이 */
    }
    .card-container { display: flex; justify-content: center; margin-top: 20px; gap: 15px; max-width: 600px; }
    .nutrition-card { background-color: #E0F7E9; padding: 15px; border-radius: 10px; width: 100px; height: 120px; text-align: center; display: flex; flex-direction: column; justify-content: center; }
    .nutrition-card h3 { margin: 0; font-size: 1em; }
    .nutrition-card p { margin: 5px 0 0; font-size: 1.2em; font-weight: bold; }
    .note { text-align: center; color: #888; font-size: 0.8em; margin-top: 10px; }
    #detailed-info-button { background-color: #FFFFFF; color: #4CAF50; font-size: 2em; font-weight: bold; border: none; cursor: pointer; padding: 10px 20px; }
    #image-upload-area { background-color: #FFFFFF; color: #4CAF50; padding: 10px; border-radius: 10px; }  /* 이미지 업로드 영역 스타일 */
"""

# CSV 파일에서 영양 성분 데이터를 로드하는 함수
def load_nutrition_data(food_name):
    df = pd.read_csv("C:/Users/최연희/Documents/yeonhui/msaischool/project/data/nutrition_data_converted.csv")
    nutrition_info = df[df['food_name'].str.lower() == food_name.lower()].to_dict('records')
    return nutrition_info[0] if nutrition_info else None

# 이미지 분석 함수
def analyze_image(image):
    try:
        image_stream = BytesIO()
        image.save(image_stream, format="JPEG")
        image_stream.seek(0)
        results = predictor.classify_image(project_id, model_name, image_stream)
       
        highest_tag = results.predictions[0].tag_name
        return highest_tag
    except Exception as e:
        return f"오류가 발생했습니다: {str(e)}"

# 권장 섭취량 계산 함수
def calculate_recommended_intake(gender, height, weight, age, activity_level):
    try:
        height = float(height)
        weight = float(weight)
        age = int(age)

        # 표준 체중 계산 (BMI 22 기준)
        standard_weight = (height / 100) ** 2 * 22
       
        # 활동 수준에 따른 계수
        activity_factors = {
            "낮음": 25,
            "보통": 30,
            "높음": 35,
            "심함": 40
        }
        activity_factor = activity_factors.get(activity_level, 25)
       
        # 권장 섭취량 계산
        recommended_intake = standard_weight * activity_factor
        return f"권장 섭취량: {recommended_intake:.2f} kcal"
    except ValueError:
        return "올바른 신체 정보를 입력해주세요."

# 두 번째 화면에서 영양 정보를 카드로 표시하고 파이차트를 생성하는 함수
def display_nutrition_info_with_chart(food_name):
    nutrition_info = load_nutrition_data(food_name)
    if nutrition_info:
        # 파이차트 생성
        fig, ax = plt.subplots()
        labels = ['Carbs', 'Protein', 'Fat']
        values = [nutrition_info['carb'], nutrition_info['protein'], nutrition_info['fat']]
        colors = ['#84B9C0', '#E26559', '#FFE697']  # 지정된 색상
        ax.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
        ax.axis('equal')
       
        return {
            "Calories": f"{nutrition_info['kcal']} kcal",
            "Carbs": f"{nutrition_info['carb']} g",
            "Protein": f"{nutrition_info['protein']} g",
            "Fat": f"{nutrition_info['fat']} g",
            "Sugars": f"{nutrition_info['sugar']} g",
            "Sodium": f"{nutrition_info['sodium']} mg"
        }, fig
    else:
        return {"Error": f"{food_name}에 대한 영양 정보가 없습니다."}, None

# Gradio Blocks 컨텍스트 내에서 코드 작성
with gr.Blocks(css=custom_css) as demo:
    # 홈 화면
    with gr.Column(visible=True) as home_screen:
        gr.Markdown("<div id='title'>FoodSync AI</div>")
        start_button = gr.Button("식단 분석하기", elem_id="start-button")

    # 첫 번째 화면
    with gr.Column(visible=False) as first_screen:
        # Image Upload 제목을 버튼으로 만들고 흰색 배경 및 초록색 글씨 설정
        upload_button = gr.Button("Image Upload", elem_id="image-upload-area")
        # 신체 정보 입력 필드 (상단)
        gender = gr.Radio(["남자", "여자"], label="성별")
        height = gr.Textbox(label="신장 (cm)", placeholder="예: 180")
        weight = gr.Textbox(label="체중 (kg)", placeholder="예: 70")
        age = gr.Textbox(label="나이 (세)", placeholder="예: 25")
        activity_level = gr.Radio(["낮음", "보통", "높음", "심함"], label="활동 수준", 
                                  info="육체활동이 거의 없는 경우 (낮음), 보통의 활동 (보통), 심한 활동 (높음), 매우 심한 활동 (심함)")

        # 권장 섭취량 출력 필드 및 버튼
        recommended_intake_output = gr.Textbox(label="권장 섭취량", interactive=False)
        calculate_button = gr.Button("권장 섭취량 계산")

        # 권장 섭취량 계산 버튼 클릭 시
        calculate_button.click(
            fn=calculate_recommended_intake,
            inputs=[gender, height, weight, age, activity_level],
            outputs=recommended_intake_output
        )

        # 이미지 업로드 및 결과 출력 필드 (중간)
        image_input = gr.Image(label="이미지를 업로드해주세요", type="pil")
        submit_button = gr.Button("Submit")
        output_text = gr.Textbox(label="분석 결과")
    

        # 첫 번째 화면에서 submit 버튼 클릭 시 이미지 분석
        def process_image(image):
            food_name = analyze_image(image)
            return food_name

        submit_button.click(
            fn=process_image,
            inputs=image_input,
            outputs=output_text
        )

        # 상세 결과 보기 버튼 (하단)
        detail_button = gr.Button("상세 결과 화면 보기")

    # 두 번째 화면 설정 (상세 정보 화면)
    with gr.Column(visible=False, elem_id="centered-content") as second_screen:
         # 상단에 "Nutrition Facts" 제목 추가
        detailed_info_button = gr.Button("Nutrition Facts", elem_id="detailed-info-button")

        gr.Markdown("<div class='note'>100g을 기준으로 작성되었습니다</div>")
        with gr.Row(elem_id="card-container"):
            kcal_card = gr.Markdown()
            carb_card = gr.Markdown()
            protein_card = gr.Markdown()
            fat_card = gr.Markdown()
            sugar_card = gr.Markdown()
            sodium_card = gr.Markdown()
        pie_chart = gr.Plot()
        recommendation_output = gr.Markdown()  # 추천 식단을 보여줄 Markdown 요소
        back_button = gr.Button("뒤로 가기")

    # 홈 화면에서 첫 번째 화면으로 이동 설정
    start_button.click(
        fn=lambda: (gr.update(visible=False), gr.update(visible=True)),
        outputs=[home_screen, first_screen]
    )

    # 상세 결과 화면 보기 버튼 클릭 시 두 번째 화면으로 이동
    def show_second_screen(food_name):
        nutrition_info, chart = display_nutrition_info_with_chart(food_name)
        
        with open('meal_recommender_model.pkl', 'rb') as f:
            loaded_model = pickle.load(f)
        
        recommendations = loaded_model['recommender'].recommend_meals(food_name)
        recommendation_text = format_recommendations(recommendations, food_name)

        if "Error" in nutrition_info:
            return (
                gr.update(visible=False), 
                gr.update(visible=True),
                nutrition_info["Error"], 
                "", "", "", "", "", "",
                ""
            )
            
        # 페이지 전환 시 요소 상태 업데이트
        return (
            gr.update(visible=False), 
            gr.update(visible=True),
            f"<div class='nutrition-card'><h3>칼로리</h3><p>{nutrition_info['Calories']}</p></div>",
            f"<div class='nutrition-card'><h3>탄수화물</h3><p>{nutrition_info['Carbs']}</p></div>",
            f"<div class='nutrition-card'><h3>단백질</h3><p>{nutrition_info['Protein']}</p></div>",
            f"<div class='nutrition-card'><h3>지방</h3><p>{nutrition_info['Fat']}</p></div>",
            f"<div class='nutrition-card'><h3>당류</h3><p>{nutrition_info['Sugars']}</p></div>",
            f"<div class='nutrition-card'><h3>나트륨</h3><p>{nutrition_info['Sodium']}</p></div>",
            chart if chart else gr.Plot(),
            recommendation_text  # 추천 식단 반환
        )

    detail_button.click(
        fn=show_second_screen,
        inputs=output_text,
        outputs=[
            first_screen, 
            second_screen, 
            kcal_card, 
            carb_card, 
            protein_card, 
            fat_card, 
            sugar_card, 
            sodium_card, 
            pie_chart,
            recommendation_output  # 추천 식단 출력 요소
        ]
    )

    # 뒤로가기 버튼 동작 설정
    def show_first_screen():
        return gr.update(visible=True), gr.update(visible=False)

    back_button.click(
        fn=show_first_screen, 
        outputs=[first_screen, second_screen]
    )

# 앱 실행
demo.launch(share=True)


C:\Users\최연희\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7884
* Running on public URL: https://e983ab956c76717efd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
